In [1]:
# Setup (Imports)

from collections import defaultdict

from datetime import datetime
from praw import Reddit
import requests

In [2]:
reddit = Reddit('StockMarketML')

articles = defaultdict(list)

for submission in reddit.subreddit('news+apple+ios+AAPL').search('apple', limit=None):
    articles[datetime.fromtimestamp(submission.created).strftime('%Y-%m-%d')].append(submission.title)
    
articles

defaultdict(list,
            {'2009-06-03': ['I love my Mac as much as the next guy, but sometimes Apple just makes really silly design choices.  What Apple bug/feature has frustrated you the most?'],
             '2011-03-21': ['Man returns iPad 2 because his "wife said no." Apple execs send him a free one with a note reading "Apple said yes."'],
             '2011-05-31': ['Apple to Unveil Lion, iOS5 and iCloud at Keynote on Monday, June 6'],
             '2011-08-25': ["Apple's Steve Jobs says resigns as CEO"],
             '2011-09-25': ['Samsung have not copied Apple. Here is the proof!'],
             '2011-12-10': ["Apple's newly released ad with past Beatles cover art - very well done Apple.\n"],
             '2012-04-18': ['Apple tries to screw the little guy. Little guy wins in court.'],
             '2012-05-11': ["iOS6 drops Google Maps and debuts Apple's own"],
             '2012-07-29': ["A simple table showing what your Apple device will and won't get in iOS 6."],
     